# **Basic Data Analytics**

## Objectives

* In this notebook, we will do some preliminary statistical analysis, such as a correlation study, as well as various visualizations.

## Inputs

* The input for this is the cleaned data from the last notebook, namely `'game_data_clean.csv'`.

## Outputs

* At the end, we will have various plots displaying the statistical relationship between different features of our dataset.

## Additional Comments

* In case you have any additional comments that don't fit in the previous bullets, please state them here. 


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [2]:
import os
home_dir = '/workspace/pp5-ml-dashboard'
csv_dir ='/workspace/pp5-ml-dashboard/outputs/datasets/clean/csv' 
os.chdir(home_dir)
current_dir = os.getcwd()
print(current_dir)



/workspace/pp5-ml-dashboard


We now load our cleaned dataset as well as some of the packages that we will be using.

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from ydata_profiling import ProfileReport
from src.utils import get_df

game_data = get_df('game_data_clean', csv_dir)
game_data.head()

,season_id,team_id_home,game_id,wl_home,min,fgm_home,fga_home,fg3m_home,fg3a_home,ftm_home,...,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,plus_minus_away,Day,Month,Year
0,21985,1610612737,28500005,L,240,41.0,92.0,0.0,3.0,9.0,...,21.0,11.0,7.0,17.0,19.0,100.0,9,25,10,1985
1,21985,1610612758,28500006,L,240,39.0,88.0,0.0,2.0,26.0,...,19.0,7.0,7.0,18.0,32.0,108.0,4,25,10,1985
2,21985,1610612765,28500010,W,240,39.0,88.0,0.0,1.0,40.0,...,27.0,10.0,7.0,20.0,32.0,116.0,-2,25,10,1985
3,21985,1610612762,28500011,L,240,42.0,82.0,0.0,2.0,24.0,...,23.0,10.0,7.0,19.0,28.0,112.0,4,25,10,1985
4,21985,1610612744,28500008,L,240,36.0,91.0,0.0,4.0,33.0,...,26.0,11.0,3.0,22.0,40.0,119.0,14,25,10,1985


# Section 1: Correlation study and visualization
We hypothesize that many of the statistics will be correlated with each other. After dropping certain categorical features, lets look the correlation dataframe.

Note: add in playoff and regular season column to see if anything correlates there, and maybe leave year in, or add season in as an ordinal thing. maybe make team ordinal as well. drop day, maybe drop month as well.

In [5]:
game_data.columns

Index(['season_id', 'team_id_home', 'game_id', 'wl_home', 'min', 'fgm_home',
       'fga_home', 'fg3m_home', 'fg3a_home', 'ftm_home', 'fta_home',
       'oreb_home', 'dreb_home', 'reb_home', 'ast_home', 'stl_home',
       'blk_home', 'tov_home', 'pf_home', 'pts_home', 'plus_minus_home',
       'team_id_away', 'wl_away', 'fgm_away', 'fga_away', 'fg3m_away',
       'fg3a_away', 'ftm_away', 'fta_away', 'oreb_away', 'dreb_away',
       'reb_away', 'ast_away', 'stl_away', 'blk_away', 'tov_away', 'pf_away',
       'pts_away', 'plus_minus_away', 'Day', 'Month', 'Year'],
      dtype='object')

Section 1 content

---

# Section 2

Section 2 content

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In case you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create here your folder
  # os.makedirs(name='')
except Exception as e:
  print(e)
